# Installing required packages

In [1]:
!pip install tensorflow

In [2]:
!pip install nltk

# Importing required libraries

In [3]:
import json
import pickle
import random
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to C:\Users\Prasham
[nltk_data]     Bhagwat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Prasham
[nltk_data]     Bhagwat\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Making lists to store words, classes and documents

In [4]:
vocabulary_list = []
categories_list = []
document_list = []

# List of characters to ignore

In [5]:
characters = ['?', '!']

# Opening and Loading the Dataset 

In [6]:
dataset = open('mentalhealth.json').read()
intents = json.loads(dataset)

# Updating the vocabulary, category and documents lists

In [7]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #Tokenizing each word from pattern into the vocabulary list
        w = nltk.word_tokenize(pattern)
        vocabulary_list.extend(w)
        
        #Adding documents into the collection (corpus)
        document_list.append((w, intent['tag']))

        #Adding unique tags into the categories list
        if intent['tag'] not in categories_list:
            categories_list.append(intent['tag'])

# Text pre-processing (lower case, remove non-alphanumeric characters and lemmatize text)

In [8]:
lemma = WordNetLemmatizer()
vocabulary = [lemma.lemmatize(w.lower()) for w in vocabulary_list if w not in characters]
vocabulary = sorted(list(set(vocabulary)))

# Sorting the categories list

In [9]:
categories = sorted(list(set(categories_list)))

# Documents list is a combination between intents and patterns

In [10]:
print (len(document_list), "documents")

180 documents


# Categories list is the intents

In [11]:
print (len(categories), "classes", categories)

57 classes ['Bhopal_support', 'ahemdabad_support', 'anxiety_causes', 'anxiety_disorders', 'anxiety_symptoms', 'anxiety_treatment', 'bangalore_support', 'bipolar_causes', 'bipolar_disorder', 'bipolar_symptoms', 'bipolar_treatment', 'chennai_support', 'delhi_support', 'depression', 'depression_causes', 'depression_symptoms', 'depression_treatment', 'eating_causes', 'eating_disorder', 'eating_symptoms', 'eating_treatment', 'generalized_disorder', 'goodbye', 'greeting', 'helpline_select', 'helpline_select_app', 'helpline_select_group', 'india_helpline', 'india_support', 'kolkata_support', 'mental_health', 'mental_health_illness', 'mental_illness', 'mental_illness_causes', 'mental_illness_symptoms', 'mental_illness_types', 'mental_treatment', 'mumbai_support', 'noanswer', 'obsessive_disorder', 'options', 'panic_disorder', 'personality_causes', 'personality_disorders', 'personality_symptoms', 'personality_treatment', 'phobias', 'post_disorder', 'post_symptoms', 'pune_support', 'schizophrenia

# Vocabulary list is all the unique words

In [12]:
print (len(vocabulary), "Unique lemmatized words", vocabulary)

122 Unique lemmatized words ["'s", ',', '1', '2', '3', '4', 'a', 'about', 'ahemdabad', 'and', 'anxiety', 'any', 'anyone', 'apps', 'are', 'available', 'awesome', 'bangalore', 'better', 'bhopal', 'bipolar', 'both', 'bye', 'call', 'can', 'care', 'cause', 'chat', 'chatting', 'chennai', 'concern', 'contact', 'could', 'crisis', 'day', 'delhi', 'depression', 'detail', 'different', 'disorder', 'do', 'eating', 'factor', 'for', 'generalized', 'get', 'give', 'good', 'goodbye', 'group', 'have', 'health', 'hello', 'help', 'helpful', 'helping', 'helpline', 'hey', 'hi', 'how', 'i', 'illness', 'in', 'india', 'information', 'is', 'issue', 'kolkata', 'later', 'me', 'mental', 'more', 'mumbai', 'my', 'next', 'nice', 'number', 'obsessive-compulsive', 'of', 'on', 'online', 'panic', 'peer', 'personality', 'phobia', 'phone', 'post-traumatic', 'provide', 'pune', 'regarding', 'region', 'region-wise', 'resource', 'same', 'schizophrenia', 'see', 'service', 'should', 'social', 'some', 'specific', 'stress', 'suppor

# Converting the vocabulary and category lists from objects into byte-stream and storing in respective pickle files

In [13]:
pickle.dump(vocabulary,open('vocabulary.pkl','wb'))
pickle.dump(categories,open('categories.pkl','wb'))

# Creating a list for the training data

In [14]:
traininglist = []

# Creating and empty array to store the outputs

In [15]:
output_empty = [0] * len(categories)

# Training set will consist of bag of words for each text sentence

In [16]:
for doc in document_list:
    # Creating the bag of words list
    bagofwords = []
    
    # Creating a list of tokenized words for the pattern
    pattern_words = doc[0]
    
    # Lemmatizing each word - Creating base lemma words in an attempt to represent relative words
    pattern_words = [lemma.lemmatize(word.lower()) for word in pattern_words]
    
    # Creating the bag of words array with 1, if word match is found in the current pattern
    for w in vocabulary:
        bagofwords.append(1) if w in pattern_words else bagofwords.append(0)
    
    # Output will be '0' for each tag and '1' will be for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[categories.index(doc[1])] = 1
    traininglist.append([bagofwords, output_row])

# Shuffling the training data and transforming the list into a numpy array

In [17]:
random.shuffle(traininglist)
training = np.array(traininglist)

<ipython-input-17-f8df48a8ad45>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(traininglist)


# Creating the training and testing lists. X is the patterns, Y is the intents

In [18]:
X_train = list(training[:,0])
Y_train = list(training[:,1])
print("Training data created")

Training data created


# Creating the Chatbot model - 3 layers. First layer contains 128 neurons, second layer contains 64 neurons and third output layer contains number of neurons that are equal to number of intents for predicting the output intent with softmax

In [19]:
# Sequential model code adapted from 'Keras' documentation
# https://keras.io/guides/sequential_model/

botmodel = Sequential()
botmodel.add(Dense(128, input_shape=(len(X_train[0]),), activation='relu'))
botmodel.add(Dropout(0.5))
botmodel.add(Dense(64, activation='relu'))
botmodel.add(Dropout(0.5))
botmodel.add(Dense(len(Y_train[0]), activation='softmax'))

# Compiling the Chatbot model.

In [20]:
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
botmodel.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fitting and saving the Chatbot model for future use

In [21]:
createmodel = botmodel.fit(np.array(X_train), np.array(Y_train), epochs=1000, batch_size=8, verbose=1)
botmodel.save('mentalhealthbot_model.h5', createmodel)
print("Model created")

Epoch 1/1000
23/23 [==============================] - 1s 1ms/step - loss: 4.0487 - accuracy: 0.0222 
Epoch 2/1000
23/23 [==============================] - 0s 1ms/step - loss: 3.9907 - accuracy: 0.0222
Epoch 3/1000
23/23 [==============================] - 0s 2ms/step - loss: 3.9480 - accuracy: 0.0444
Epoch 4/1000
23/23 [==============================] - 0s 1ms/step - loss: 3.8900 - accuracy: 0.0556
Epoch 5/1000
23/23 [==============================] - 0s 1ms/step - loss: 3.8347 - accuracy: 0.0667
Epoch 6/1000
23/23 [==============================] - 0s 1ms/step - loss: 3.7539 - accuracy: 0.0778
Epoch 7/1000
23/23 [==============================] - 0s 997us/step - loss: 3.6772 - accuracy: 0.0944
Epoch 8/1000
23/23 [==============================] - 0s 997us/step - loss: 3.6063 - accuracy: 0.1167
Epoch 9/1000
23/23 [==============================] - 0s 952us/step - loss: 3.4304 - accuracy: 0.1611
Epoch 10/1000
23/23 [==============================] - 0s 1ms/step - loss: 3.3549 - accuracy:

23/23 [==============================] - 0s 725us/step - loss: 0.5364 - accuracy: 0.8389
Epoch 81/1000
23/23 [==============================] - 0s 725us/step - loss: 0.5842 - accuracy: 0.8056
Epoch 82/1000
23/23 [==============================] - 0s 680us/step - loss: 0.5505 - accuracy: 0.8222
Epoch 83/1000
23/23 [==============================] - 0s 907us/step - loss: 0.5246 - accuracy: 0.8389
Epoch 84/1000
23/23 [==============================] - 0s 771us/step - loss: 0.4914 - accuracy: 0.8333
Epoch 85/1000
23/23 [==============================] - 0s 771us/step - loss: 0.3962 - accuracy: 0.8833
Epoch 86/1000
23/23 [==============================] - 0s 725us/step - loss: 0.5371 - accuracy: 0.8389
Epoch 87/1000
23/23 [==============================] - 0s 725us/step - loss: 0.4768 - accuracy: 0.8667
Epoch 88/1000
23/23 [==============================] - 0s 725us/step - loss: 0.5687 - accuracy: 0.8222
Epoch 89/1000
23/23 [==============================] - 0s 771us/step - loss: 0.4271 - a

23/23 [==============================] - 0s 771us/step - loss: 0.2685 - accuracy: 0.9444
Epoch 160/1000
23/23 [==============================] - 0s 816us/step - loss: 0.4096 - accuracy: 0.8667
Epoch 161/1000
23/23 [==============================] - 0s 861us/step - loss: 0.2001 - accuracy: 0.9389
Epoch 162/1000
23/23 [==============================] - 0s 816us/step - loss: 0.3083 - accuracy: 0.8944
Epoch 163/1000
23/23 [==============================] - 0s 724us/step - loss: 0.1467 - accuracy: 0.9667
Epoch 164/1000
23/23 [==============================] - 0s 725us/step - loss: 0.2663 - accuracy: 0.9167
Epoch 165/1000
23/23 [==============================] - 0s 725us/step - loss: 0.3533 - accuracy: 0.8722
Epoch 166/1000
23/23 [==============================] - 0s 816us/step - loss: 0.3534 - accuracy: 0.8833
Epoch 167/1000
23/23 [==============================] - 0s 725us/step - loss: 0.2316 - accuracy: 0.9333
Epoch 168/1000
23/23 [==============================] - 0s 816us/step - loss: 0

23/23 [==============================] - 0s 816us/step - loss: 0.2359 - accuracy: 0.9000
Epoch 239/1000
23/23 [==============================] - 0s 998us/step - loss: 0.2439 - accuracy: 0.9222
Epoch 240/1000
23/23 [==============================] - 0s 997us/step - loss: 0.1252 - accuracy: 0.9667
Epoch 241/1000
23/23 [==============================] - 0s 771us/step - loss: 0.1861 - accuracy: 0.9444
Epoch 242/1000
23/23 [==============================] - 0s 771us/step - loss: 0.2265 - accuracy: 0.9222
Epoch 243/1000
23/23 [==============================] - 0s 771us/step - loss: 0.2874 - accuracy: 0.8889
Epoch 244/1000
23/23 [==============================] - 0s 725us/step - loss: 0.2524 - accuracy: 0.9000
Epoch 245/1000
23/23 [==============================] - 0s 952us/step - loss: 0.1951 - accuracy: 0.9500
Epoch 246/1000
23/23 [==============================] - 0s 952us/step - loss: 0.2633 - accuracy: 0.9111
Epoch 247/1000
23/23 [==============================] - 0s 1ms/step - loss: 0.2

23/23 [==============================] - 0s 4ms/step - loss: 0.1236 - accuracy: 0.9611
Epoch 318/1000
23/23 [==============================] - 0s 4ms/step - loss: 0.1794 - accuracy: 0.9278
Epoch 319/1000
23/23 [==============================] - 0s 4ms/step - loss: 0.1938 - accuracy: 0.9389
Epoch 320/1000
23/23 [==============================] - 0s 4ms/step - loss: 0.1817 - accuracy: 0.9278
Epoch 321/1000
23/23 [==============================] - ETA: 0s - loss: 0.1793 - accuracy: 0.91 - 0s 4ms/step - loss: 0.1861 - accuracy: 0.9278
Epoch 322/1000
23/23 [==============================] - 0s 4ms/step - loss: 0.1781 - accuracy: 0.9500
Epoch 323/1000
23/23 [==============================] - 0s 4ms/step - loss: 0.2166 - accuracy: 0.9167
Epoch 324/1000
23/23 [==============================] - 0s 5ms/step - loss: 0.1843 - accuracy: 0.9333
Epoch 325/1000
23/23 [==============================] - 0s 5ms/step - loss: 0.1726 - accuracy: 0.9333
Epoch 326/1000
23/23 [==============================] -

...

23/23 [==============================] - 0s 2ms/step - loss: 0.1721 - accuracy: 0.9500
Epoch 637/1000
23/23 [==============================] - 0s 2ms/step - loss: 0.1328 - accuracy: 0.9500
Epoch 638/1000
23/23 [==============================] - 0s 3ms/step - loss: 0.1627 - accuracy: 0.9389
Epoch 639/1000
23/23 [==============================] - 0s 2ms/step - loss: 0.0583 - accuracy: 0.9722
Epoch 640/1000
23/23 [==============================] - 0s 2ms/step - loss: 0.1435 - accuracy: 0.9500
Epoch 641/1000
23/23 [==============================] - 0s 3ms/step - loss: 0.2090 - accuracy: 0.9389
Epoch 642/1000
23/23 [==============================] - 0s 2ms/step - loss: 0.1827 - accuracy: 0.9333
Epoch 643/1000
23/23 [==============================] - 0s 2ms/step - loss: 0.1141 - accuracy: 0.9500
Epoch 644/1000
23/23 [==============================] - 0s 2ms/step - loss: 0.1451 - accuracy: 0.9389
Epoch 645/1000
23/23 [==============================] - 0s 3ms/step - loss: 0.1228 - accuracy: 0.

23/23 [==============================] - 0s 3ms/step - loss: 0.1257 - accuracy: 0.9556
Epoch 716/1000
23/23 [==============================] - 0s 2ms/step - loss: 0.1068 - accuracy: 0.9500
Epoch 717/1000
23/23 [==============================] - 0s 2ms/step - loss: 0.0790 - accuracy: 0.9778
Epoch 718/1000
23/23 [==============================] - 0s 2ms/step - loss: 0.2963 - accuracy: 0.9000
Epoch 719/1000
23/23 [==============================] - 0s 3ms/step - loss: 0.0662 - accuracy: 0.9667
Epoch 720/1000
23/23 [==============================] - 0s 3ms/step - loss: 0.1737 - accuracy: 0.9444
Epoch 721/1000
23/23 [==============================] - 0s 3ms/step - loss: 0.1833 - accuracy: 0.9389
Epoch 722/1000
23/23 [==============================] - 0s 2ms/step - loss: 0.1333 - accuracy: 0.9556
Epoch 723/1000
23/23 [==============================] - 0s 2ms/step - loss: 0.2027 - accuracy: 0.9500
Epoch 724/1000
23/23 [==============================] - 0s 4ms/step - loss: 0.0992 - accuracy: 0.

Epoch 795/1000
23/23 [==============================] - 0s 952us/step - loss: 0.1046 - accuracy: 0.9667
Epoch 796/1000
23/23 [==============================] - 0s 1ms/step - loss: 0.1151 - accuracy: 0.9556
Epoch 797/1000
23/23 [==============================] - 0s 1ms/step - loss: 0.0922 - accuracy: 0.9722
Epoch 798/1000
23/23 [==============================] - 0s 952us/step - loss: 0.0645 - accuracy: 0.9778
Epoch 799/1000
23/23 [==============================] - 0s 907us/step - loss: 0.1554 - accuracy: 0.9389
Epoch 800/1000
23/23 [==============================] - 0s 1ms/step - loss: 0.1506 - accuracy: 0.9444
Epoch 801/1000
23/23 [==============================] - 0s 1ms/step - loss: 0.1272 - accuracy: 0.9444
Epoch 802/1000
23/23 [==============================] - 0s 1ms/step - loss: 0.0747 - accuracy: 0.9556
Epoch 803/1000
23/23 [==============================] - 0s 2ms/step - loss: 0.1231 - accuracy: 0.9556
Epoch 804/1000
23/23 [==============================] - 0s 1ms/step - loss: 

23/23 [==============================] - 0s 1ms/step - loss: 0.1630 - accuracy: 0.9389
Epoch 875/1000
23/23 [==============================] - 0s 896us/step - loss: 0.1301 - accuracy: 0.9278
Epoch 876/1000
23/23 [==============================] - 0s 1ms/step - loss: 0.1702 - accuracy: 0.9333
Epoch 877/1000
23/23 [==============================] - 0s 1ms/step - loss: 0.1441 - accuracy: 0.9611
Epoch 878/1000
23/23 [==============================] - 0s 997us/step - loss: 0.1023 - accuracy: 0.9611
Epoch 879/1000
23/23 [==============================] - 0s 952us/step - loss: 0.2015 - accuracy: 0.9500
Epoch 880/1000
23/23 [==============================] - 0s 861us/step - loss: 0.2230 - accuracy: 0.9167
Epoch 881/1000
23/23 [==============================] - 0s 1ms/step - loss: 0.1117 - accuracy: 0.9611
Epoch 882/1000
23/23 [==============================] - 0s 1ms/step - loss: 0.1012 - accuracy: 0.9444
Epoch 883/1000
23/23 [==============================] - 0s 1ms/step - loss: 0.1200 - accu

23/23 [==============================] - 0s 2ms/step - loss: 0.2500 - accuracy: 0.8889
Epoch 954/1000
23/23 [==============================] - 0s 2ms/step - loss: 0.1834 - accuracy: 0.9333
Epoch 955/1000
23/23 [==============================] - 0s 2ms/step - loss: 0.0871 - accuracy: 0.9667
Epoch 956/1000
23/23 [==============================] - 0s 2ms/step - loss: 0.0828 - accuracy: 0.9722
Epoch 957/1000
23/23 [==============================] - 0s 2ms/step - loss: 0.1279 - accuracy: 0.9333
Epoch 958/1000
23/23 [==============================] - 0s 2ms/step - loss: 0.1876 - accuracy: 0.9278
Epoch 959/1000
23/23 [==============================] - 0s 2ms/step - loss: 0.0808 - accuracy: 0.9722
Epoch 960/1000
23/23 [==============================] - 0s 2ms/step - loss: 0.0663 - accuracy: 0.9722
Epoch 961/1000
23/23 [==============================] - 0s 2ms/step - loss: 0.0993 - accuracy: 0.9556
Epoch 962/1000
23/23 [==============================] - 0s 2ms/step - loss: 0.0862 - accuracy: 0.

# Summary of the model

In [22]:
botmodel.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               15744     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 57)                3705      
Total params: 27,705
Trainable params: 27,705
Non-trainable params: 0
_________________________________________________________________
